In [29]:
import os
import numpy as np
import re
import shutil
import tensorflow as tf
import nltk
import numpy as np
import re
import shutil
import tensorflow as tf
import os
import unicodedata
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, TimeDistributed
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [4]:
#!/bin/bash
!curl -L -o english-to-german.zip  https://www.kaggle.com/api/v1/datasets/download/kaushal2896/english-to-german
!unzip english-to-german.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 8184k  100 8184k    0     0  11.2M      0 --:--:-- --:--:-- --:--:-- 30.6M
Archive:  english-to-german.zip
  inflating: _about.txt              
  inflating: deu.txt                 


In [24]:
def preprocess_sentence(sent):
    sent = "".join([c for c in unicodedata.normalize("NFD", sent)
        if unicodedata.category(c) != "Mn"])
    sent = re.sub(r"([!.?])", r" \1", sent)
    sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)
    sent = re.sub(r"\s+", " ", sent)
    sent = sent.lower()
    return sent

def read_data(filepath, num_sent_pairs):
    en_sents, target_lang_sents = [], []
    with open(filepath, "r") as fin:
        for i, line in enumerate(fin):
            en_sent, target_lang_sent, _ = line.strip().split('\t')
            en_sent = [w for w in preprocess_sentence(en_sent).split()]
            target_lang_sent = preprocess_sentence(target_lang_sent)
            target_lang_sent = [w for w in preprocess_sentence(target_lang_sent).split()]
            en_sents.append(en_sent)
            target_lang_sents.append(target_lang_sent)
            if i >= num_sent_pairs - 1:
                break
    return en_sents, target_lang_sents

def vocab_builder(sents):
    vocab = set()
    for sent in sents:
        for word in sent:
            vocab.add(word)
    return vocab

In [30]:
en_sents, target_lang_sents  = read_data("/content/deu.txt", 500)

en_vocab = vocab_builder(en_sents)
target_lang_vocab = vocab_builder(target_lang_sents)

print("en_vocab size: {:d}".format(len(en_vocab)))
print("target_lang_vocab size: {:d}".format(len(target_lang_vocab)))

EN_VOCAB_SIZE = len(en_vocab)
DE_VOCAB_SIZE = len(target_lang_vocab)

MAX_SEQ_LENGTH = max(map(len, en_sents))
EMBEDDING_DIM = 128

en_vocab size: 210
target_lang_vocab size: 397


In [34]:
# prompt: tokenize en_sents, target_lang_sents

from nltk.tokenize import word_tokenize
nltk.download('punkt')

en_tokenized = []
for sent in en_sents:
    en_tokenized.append(word_tokenize(" ".join(sent)))

target_lang_tokenized = []
for sent in target_lang_sents:
    target_lang_tokenized.append(word_tokenize(" ".join(sent)))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [32]:
model = Sequential([
    Embedding(input_dim=EN_VOCAB_SIZE, output_dim=EMBEDDING_DIM, input_length=MAX_SEQ_LENGTH),
    SimpleRNN(128, return_sequences=True),
    TimeDistributed(Dense(DE_VOCAB_SIZE, activation='softmax'))
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# y_train must be 3D (samples, sequence, 1) for sparse_categorical_crossentropy
# y_train = np.expand_dims(y_train, -1)

# # Train the model
model.fit(en_sents, target_lang_sents, batch_size=64, epochs=10, validation_split=0.2)


ValueError: Argument `validation_split` is only supported for tensors or NumPy arrays.Found incompatible type in the input: [<class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>, <clas